In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from utils.TD import TD

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

np.random.seed(42)

In [4]:
housedata = np.loadtxt('data\\readyhousedata.txt', delimiter=',')

X = housedata[:, :-1]
y = housedata[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = LinearRegression() 
reg.fit(X_train, y_train)

weights = reg.coef_
intercept = reg.intercept_

print(weights)
print("Intercept:", intercept)

score = reg.score(X_test, y_test)
print("Model R^2 score:", score)

[-10.99366542   5.00578355   0.63581661   2.44872684 -10.31186138
  17.51763825   1.03114679 -17.77574728   7.34087816  -6.42763345
  -8.79966234   3.03563353 -20.51935608]
Intercept: 29.04431567479631
Model R^2 score: 0.7730570034661727


In [5]:
num_samples = X_train.shape[0]
P = np.ones((num_samples, num_samples)) / num_samples # Equal probability to move to any state

alpha = 0.01  # Learning rate
gamma = 0   # Discount factor
num_iterations = 1e6  # Number of iterations
epsilon = 1e-9

td = TD(
    n_iter=num_iterations,
    P=P,
    link=lambda x : x,
    inv_link=lambda x : x,
    gamma=gamma,
    alpha=alpha,
    epsilon=epsilon,
)

td.fit(X_train, y_train)

w_hat_house = td.weights
bias_house = td.bias

print(w_hat_house)

[-11.18211572   5.11801614   0.48564318   1.96249423  -9.94041809
  17.75437566   0.74035574 -17.16481025   7.18176397  -6.59126913
  -9.10714929   3.24368026 -20.77519175]


In [6]:
def mini_batch_sgd(X: np.ndarray, y: np.ndarray, learning_rate: float, n_iter: int, epsilon: float, batch_size: int) -> np.ndarray:
    """
    Performs Mini-Batch Stochastic Gradient Descent (SGD) for linear regression with shuffling.
    
    Parameters:
    X (np.ndarray): Feature matrix (n_samples, n_features).
    y (np.ndarray): Target vector (n_samples,).
    learning_rate (float): Step size for updating weights.
    n_iter (int): Number of iterations (epochs).
    epsilon (float): Convergence threshold.
    batch_size (int): Size of the mini-batches.
    
    Returns:
    np.ndarray: The learned weights.
    """
    n_samples, n_features = X.shape
    # Add bias term to the feature matrix
    X_bias = np.c_[np.ones(n_samples), X]  # Adds a column of ones for the bias term

    # Initialize weights to zeros
    weights = np.zeros(n_features + 1)

    for epoch in range(int(n_iter)):
        # Step 1: Shuffle the data at the beginning of each epoch
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        X_bias_shuffled = X_bias[indices]
        y_shuffled = y[indices]

        for i in range(0, n_samples, batch_size):
            # Step 2: Select the mini-batch
            X_batch = X_bias_shuffled[i:i + batch_size]
            y_batch = y_shuffled[i:i + batch_size]
            
            # Initialize gradients to zero
            gradient = np.zeros_like(weights)
            
            # Step 3: Compute the gradient over the mini-batch
            for j in range(X_batch.shape[0]):
                # Prediction
                prediction = np.dot(X_batch[j], weights)
                error = y_batch[j] - prediction
                
                # Update gradient
                gradient += -2 * X_batch[j] * error

            # Step 4: Update the weights
            weights -= learning_rate * gradient / batch_size

        # Optional: Check for convergence (if gradient is small enough)
        if np.linalg.norm(gradient) < epsilon:
            print(f"Converged after {epoch + 1} epochs")
            break

    return weights

# Hyperparameters
learning_rate = 0.01
n_iter = 1e4  # Number of epochs
epsilon = 1e-6  # Convergence threshold
batch_size = 16  # Mini-batch size

# Run SGD manually
weights_sgd = mini_batch_sgd(X_train, y_train, learning_rate, n_iter, epsilon, batch_size)

# Print the learned weights
print("Weights learned using SGD:\n", weights_sgd[1:])


Weights learned using SGD:
 [-10.99550082   5.00148479   0.62866441   2.44291755 -10.32353625
  17.49539486   1.02345406 -17.77975053   7.34330163  -6.42853038
  -8.8058957    3.0204479  -20.5263245 ]


In [7]:
pred_TD = td.predict(X_test)
pred_L2 = reg.predict(X_test)
pred_sgd = np.dot(X_test, weights_sgd[1:]) + weights_sgd[0]

rmse_TD = td.rmse(X_test, y_test)
rmse_L2 = np.sqrt(mean_squared_error(y_test, pred_L2))
rmse_sgd = np.sqrt(mean_squared_error(y_test, pred_sgd))

print("RMSE on the test set using TD:", rmse_TD)
print("RMSE on the test set using L2 Regression:", rmse_L2)
print("RMSE on the test set using SGD:", rmse_sgd)
print("---------------")
print("Norm of weights from TD:", np.linalg.norm(w_hat_house, 2))
print("Norm of weights from L2 Regression:", np.linalg.norm(weights, 2))
print("Norm of weights from SGD:", np.linalg.norm(weights_sgd[1:], 2))
print("Norm of difference in weights for L2:", np.linalg.norm(weights - w_hat_house, 2))
print("Norm of difference in weights for sgd:", np.linalg.norm(w_hat_house - weights_sgd[1:], 2))


RMSE on the test set using TD: 4.098375087372361
RMSE on the test set using L2 Regression: 4.154652986817322
RMSE on the test set using SGD: 4.151977290028692
---------------
Norm of weights from TD: 38.530798371766224
Norm of weights from L2 Regression: 38.54163631701667
Norm of weights from SGD: 38.540331685087516
Norm of difference in weights for L2: 1.1019435590742406
Norm of difference in weights for sgd: 1.1076609311472736
